In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [128]:
data = pd.read_csv("total_data.csv")
data

,timestamp,submit_id,user_id,problem_id,result_state,memory,running_time,language,code_length,__
0,2022-12-16 17:08:57,52731042.000000,jsh00325,14562.000000,맞았습니다!!,2020.000000,0.000000,C++11,592.000000,8일 전
1,2022-12-16 15:28:07,52726708.000000,teddybear1024,14562.000000,맞았습니다!!,2024.000000,0.000000,C++17,875.000000,8일 전
2,2022-12-15 01:15:34,52680839.000000,mujigae,14562.000000,맞았습니다!!,2156.000000,0.000000,C++17,554.000000,9일 전
3,2022-12-12 15:23:46,52595252.000000,wnsgml7267,14562.000000,맞았습니다!!,34096.000000,68.000000,Python 3,420.000000,12일 전
4,2022-12-01 00:29:05,52271216.000000,gustjs5419,14562.000000,맞았습니다!!,2020.000000,4.000000,C++17,712.000000,23일 전
...,...,...,...,...,...,...,...,...,...,...
22608665,2014-09-15 00:31:26,323956.000000,yukariko,10158.000000,틀렸습니다,NaN,NaN,C99,476.000000,8년 전
22608666,2014-09-15 00:27:45,323955.000000,yukariko,10158.000000,시간 초과,NaN,NaN,C99,476.000000,8년 전
22608667,2014-09-15 00:21:24,323950.000000,yukariko,10158.000000,시간 초과,NaN,NaN,C99,369.000000,8년 전
22608668,2014-09-14 23:48:00,323934.000000,hujub,10158.000000,맞았습니다!!,1116.000000,0.000000,C++98,475.000000,8년 전


In [129]:
data_drop_na = data.drop(data.loc[data['timestamp'].isna()].index)
data_drop_na.reset_index(inplace=True, drop=True)
data_drop_na = data_drop_na.drop(data_drop_na.loc[data_drop_na['user_id'].isna()].index)
data_drop_na.reset_index(inplace=True, drop=True)

drop_idx = data_drop_na.loc[data_drop_na['code_length'].isna()].index
data_drop_na = data_drop_na.drop(drop_idx)
data_drop_na.reset_index(inplace=True, drop=True)
data_drop_na.isna().sum()

timestamp              0
submit_id              0
user_id                0
problem_id             0
result_state           0
memory          13207832
running_time    13207832
language               0
code_length            0
__                     0
dtype: int64

In [130]:
tmp = data_drop_na.copy()
tmp['result_state'] = tmp['result_state'].apply(lambda x: '맞았습니다!!' if x.endswith('점') and float(x[:-1]) >= 100 else x)
tmp2 = tmp.copy()
tmp2['result_state'] = tmp2['result_state'].apply(lambda x: '틀렸습니다' if x.endswith('점') and float(x[:-1]) < 100 else x)
tmp3 = tmp2.copy()
tmp3['result_state'] = tmp3['result_state'].apply(lambda x: '런타임 에러' if x.startswith('런타임') else x)
tmp4 = tmp3.copy()
tmp4['result_state'] = tmp4['result_state'].apply(lambda x: '틀렸습니다' if x.startswith('맞았습니다!!\xa0(') else x)
tmp4['result_state'] = tmp4['result_state'].apply(lambda x: '맞았습니다' if x.startswith('맞') else x)

clean_data = tmp4.loc[~tmp4['result_state'].isin(['채점 불가', '채점 준비 중', '채점 중', '삭제된 제출'])]
tmp['result_state'].nunique(), tmp2['result_state'].nunique(), tmp3['result_state'].nunique(), tmp4['result_state'].nunique(), clean_data['result_state'].nunique()

(807, 466, 352, 12, 8)

In [131]:
# 맞았으면서 메모리값이 없는 경우는 없음
clean_data.loc[(clean_data['memory'].isna()) & (clean_data['result_state'].str.startswith('맞'))]

,timestamp,submit_id,user_id,problem_id,result_state,memory,running_time,language,code_length,__


In [132]:
total_interaction = clean_data.problem_id.nunique() * clean_data.user_id.nunique()
sparsity = 100 - (clean_data.shape[0] / total_interaction)
sparsity

99.94256775089148

In [133]:
answer_ratio = clean_data.loc[(clean_data['result_state'].str.startswith('맞'))].shape[0] / clean_data.shape[0] * 100
wrong_ratio = 100 - answer_ratio
answer_ratio, wrong_ratio

(38.0239822135952, 61.9760177864048)

In [134]:
13207484 / clean_data.shape[0]

0.6150860306493211

In [135]:
# 메모리, 러닝타임 NAN 처리 해야함
clean_data.loc[(clean_data['memory'].isna())]

,timestamp,submit_id,user_id,problem_id,result_state,memory,running_time,language,code_length,__
5,2022-12-01 00:09:18,52270751.000000,gustjs5419,14562.000000,메모리 초과,NaN,NaN,C++17,669.000000,23일 전
11,2022-11-14 00:37:19,51673887.000000,nukec,14562.000000,메모리 초과,NaN,NaN,PyPy3,553.000000,1달 전
14,2022-11-05 22:16:52,51363074.000000,strickland,14562.000000,런타임 에러,NaN,NaN,Python 3,713.000000,1달 전
15,2022-11-05 22:12:45,51362936.000000,strickland,14562.000000,메모리 초과,NaN,NaN,PyPy3,675.000000,1달 전
17,2022-10-27 08:34:41,50998593.000000,dksktpq852,14562.000000,메모리 초과,NaN,NaN,PyPy3,478.000000,1달 전
...,...,...,...,...,...,...,...,...,...,...
21472921,2014-09-15 00:45:11,323961.000000,yukariko,10158.000000,시간 초과,NaN,NaN,C99,428.000000,8년 전
21472922,2014-09-15 00:39:05,323959.000000,yukariko,10158.000000,틀렸습니다,NaN,NaN,C99,528.000000,8년 전
21472923,2014-09-15 00:31:26,323956.000000,yukariko,10158.000000,틀렸습니다,NaN,NaN,C99,476.000000,8년 전
21472924,2014-09-15 00:27:45,323955.000000,yukariko,10158.000000,시간 초과,NaN,NaN,C99,476.000000,8년 전


In [136]:
clean_data['result_state'].unique()

array(['맞았습니다', '메모리 초과', '런타임 에러', '틀렸습니다', '컴파일 에러', '출력 초과', '시간 초과',
       '출력 형식이 잘못되었습니다'], dtype=object)

In [137]:
def integrate_language(df, col):
    df.loc[df['language'].str.startswith(col), 'language'] = col
    print(df.loc[df['language'].str.startswith(col), 'language'].unique())
    return df
integrate_lang = ['C++', 'Python', 'Java', 'C#', 'Kotlin', 'Rust', 'Ruby']

for lang in integrate_lang:
    clean_data = integrate_language(clean_data, lang)

clean_data.loc[(clean_data['language'].str.startswith('C99')) | (clean_data['language'].str.startswith('C90') | (clean_data['language'].str.startswith('C11'))), 'language'] = 'C'
clean_data.loc[clean_data['language'].str.startswith('C'), 'language'].unique()

# clean_data.loc[clean_data['language'].str.startswith('C++'), 'language'] = 'C++'
# clean_data.loc[clean_data['language'].str.startswith('C++'), 'language'].unique()
# clean_data.loc[clean_data['language'].str.startswith('Py'), 'language'] = 'Python'
# clean_data.loc[clean_data['language'].str.startswith('Py'), 'language'].unique()
# clean_data.loc[clean_data['language'].str.startswith('Java'), 'language'] = 'Java'
# clean_data.loc[clean_data['language'].str.startswith('Java'), 'language'].unique()
# clean_data.loc[clean_data['language'].str.startswith('C#'), 'language'] = 'C#'
# clean_data.loc[clean_data['language'].str.startswith('C#'), 'language'].unique()
# clean_data.loc[clean_data['language'].str.startswith('Kotlin'), 'language'] = 'Kotlin'
# clean_data.loc[clean_data['language'].str.startswith('Kotlin'), 'language'].unique()
# clean_data.loc[clean_data['language'].str.startswith('Rust'), 'language'] = 'Rust'
# clean_data.loc[clean_data['language'].str.startswith('Rust'), 'language'].unique()

['C++']
['Python']
['Java']
['C#']
['Kotlin']
['Rust']
['Ruby']


array(['C++', 'C2x', 'C', 'C#', 'Cobra', 'C2x (Clang)', 'Clojure',
       'Cobol'], dtype=object)

In [81]:
lang_cnt_df = pd.concat([pd.DataFrame(clean_data['language'].value_counts().index), pd.DataFrame(clean_data['language'].value_counts().values)], axis=1)
lang_cnt_df.columns=['language', 'count']
lang_cnt_df.to_csv("lang_cnt_df2.csv",index=False)

In [138]:
lang_value_cnt = clean_data['language'].value_counts().to_frame()
other_langs = lang_value_cnt.loc[lang_value_cnt['language'] < 7000].index
other_langs

Index(['Ruby', 'PyPy2', 'PHP', 'TypeScript', 'Text', 'D', 'C2x', 'Lua', 'Perl',
       'Pascal', 'OCaml', 'F#', 'Golfscript', 'Scala', '아희', 'F# (Mono)',
       'Fortran', 'C2x (Clang)', 'awk', 'Haskell', 'R', 'Scheme',
       'Assembly (32bit)', 'Bash', 'Assembly (64bit)', 'Befunge',
       'Visual Basic', 'VB.NET 4.0 (Mono)', 'Objective-C++', 'Clojure',
       'Objective-C', '엄준식', 'Tcl', 'Go (gccgo)', 'Brainf**k', 'D (LDC)',
       'Haxe', 'INTERCAL', 'LOLCODE', 'Rhino', 'sed', 'Groovy', 'Whitespace',
       'VB.NET 2.0 (Mono)', 'SystemVerilog', 'SpiderMonkey', 'Ada', 'Cobra',
       'bc', 'Pike', 'Cobol', 'Boo', 'Nemerle'],
      dtype='object')

In [139]:
clean_data.loc[clean_data['language'].isin(other_langs), 'language'] = 'others'
clean_data['language'].value_counts()

C++        10211132
Python      4128611
Java        3941102
PyPy3       1408985
C           1174754
node.js      234266
Kotlin       107876
Swift         92852
C#            85498
others        34810
Go            30863
Rust          21831
Name: language, dtype: int64

In [95]:
clean_data.isna().sum()

timestamp              0
submit_id              0
user_id                0
problem_id             0
result_state           0
memory          13207484
running_time    13207484
language               0
code_length            0
__                     0
dtype: int64

In [140]:
pd.set_option('display.max_colwidth', None)
pd.set_option('float_format', '{:f}'.format)
problem_lang_memo_running_mean = clean_data.groupby(by=['problem_id', 'language'])[['memory', 'running_time']]
problem_lang_memo_running_mean

In [141]:
clean_data['memory_dummy'] = clean_data['memory']
clean_data['running_time_dummy'] = clean_data['running_time']
clean_data

/var/folders/04/k2hbyvy548bbb33607hffd6r0000gn/T/ipykernel_52493/631903840.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_data['memory_dummy'] = clean_data['memory']
/var/folders/04/k2hbyvy548bbb33607hffd6r0000gn/T/ipykernel_52493/631903840.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_data['running_time_dummy'] = clean_data['running_time']


,timestamp,submit_id,user_id,problem_id,result_state,memory,running_time,language,code_length,__,memory_dummy,running_time_dummy
0,2022-12-16 17:08:57,52731042.000000,jsh00325,14562.000000,맞았습니다,2020.000000,0.000000,C++,592.000000,8일 전,2020.000000,0.000000
1,2022-12-16 15:28:07,52726708.000000,teddybear1024,14562.000000,맞았습니다,2024.000000,0.000000,C++,875.000000,8일 전,2024.000000,0.000000
2,2022-12-15 01:15:34,52680839.000000,mujigae,14562.000000,맞았습니다,2156.000000,0.000000,C++,554.000000,9일 전,2156.000000,0.000000
3,2022-12-12 15:23:46,52595252.000000,wnsgml7267,14562.000000,맞았습니다,34096.000000,68.000000,Python,420.000000,12일 전,34096.000000,68.000000
4,2022-12-01 00:29:05,52271216.000000,gustjs5419,14562.000000,맞았습니다,2020.000000,4.000000,C++,712.000000,23일 전,2020.000000,4.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
21472923,2014-09-15 00:31:26,323956.000000,yukariko,10158.000000,틀렸습니다,NaN,NaN,C,476.000000,8년 전,NaN,NaN
21472924,2014-09-15 00:27:45,323955.000000,yukariko,10158.000000,시간 초과,NaN,NaN,C,476.000000,8년 전,NaN,NaN
21472925,2014-09-15 00:21:24,323950.000000,yukariko,10158.000000,시간 초과,NaN,NaN,C,369.000000,8년 전,NaN,NaN
21472926,2014-09-14 23:48:00,323934.000000,hujub,10158.000000,맞았습니다,1116.000000,0.000000,C++,475.000000,8년 전,1116.000000,0.000000


In [121]:
dic_mean = {}
dic_runtime = {}
memory_list = []
running_time_list = []
clean_data['memory_dummy'] = clean_data['memory']
clean_data['running_time_dummy'] = clean_data['running_time']
for _, grouped in tqdm(problem_lang_memo_running_mean):
    problem_id, language = _
    memory, running_time = grouped.mean()
    dic_mean[f'(clean_data["problem_id"]=={problem_id}) & (clean_data["language"]=={language}) & (clean_data["memory"].isna())'] = memory
    dic_runtime[f'(clean_data["problem_id"]=={problem_id}) & (clean_data["language"]=={language}) & (clean_data["running_time"].isna())'] = running_time
    # clean_data.loc[(clean_data['problem_id']==problem_id) & (clean_data['language']==language) & (clean_data['memory'].isna())] = memory
    # clean_data.loc[(clean_data['problem_id']==problem_id) & (clean_data['language']==language) & (clean_data['running_time'].isna())] = running_time


100%|██████████| 18440/18440 [00:03<00:00, 5184.56it/s]


In [ ]:
for condition in tqdm(dic_mean):

True

In [133]:
# mapping dicts

# user to idx & idx to user 
user2idx = {user : idx for idx, user in enumerate(clean_data['user_id'].unique())}
idx2user = {idx : user for user, idx in user2idx.items()}
# problem_id to idx & idx to problem_id
problem2idx = {int(problem) : idx for idx, problem in enumerate(clean_data['problem_id'].unique())}
idx2problem = {idx : problem_id for problem_id, idx in problem2idx.items()}
# result state to idx & idx to result state
state2idx = {state : idx for idx, state in enumerate(clean_data['result_state'].unique())}
idx2state = {idx : state for state, idx in state2idx.items()}
# submit_id to idx & idx to submit_id
submit2idx = {int(submit) : idx for idx, submit in enumerate(sorted(clean_data['submit_id'].unique()))}
idx2submit = {idx : submit for submit, idx in submit2idx.items()}